In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import importlib
import os
import glob

import sys
sys.path.append('/home/rwollman/MyProjects/AH/Repos/dredFISH')
from dredFISH.Analysis.TissueGraph import *
from dredFISH.Analysis.Classification import *
from dredFISH.Visualization.Viz import *
from dredFISH.Analysis import Classification
from dredFISH.Utils import geomu, fileu

plt.rcParams['figure.facecolor'] = 'white' 

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
pth = os.getcwd()
parts = os.path.split(pth)
parts2 = os.path.split(parts[0])
parts3 = os.path.split(parts2[0])
inputpath = os.path.join(parts[0],parts3[1])

In [3]:
config = fileu.load_config_module(inputpath)
hybes = [c[0] for c in config.bitmap if c[0]!= 'PolyT']

AttributeError: module 'dredFISH.Utils.fileu' has no attribute 'load_config_module'

In [ ]:
TMG = TissueMultiGraph(inputpath = inputpath,redo = True)
TMG.create_cell_layer(build_spatial_graph = False,build_feature_graph = False,hybes=hybes)

In [ ]:
XY = TMG.Layers[0].XY
in_large_comp = geomu.in_graph_large_connected_components(XY,large_comp_def = 0.05,plot_comp = True)

In [ ]:
print(f"before filtering: {TMG.N[0]} cells")
TMG.Layers[0].filter(in_large_comp)
print(f"after filtering: {TMG.N[0]} cells")

In [ ]:
vmin,vmax = np.percentile(TMG.Layers[0].adata.obs['dapi'],[1,99])
vmin,vmax

In [ ]:
DapiValueDistributions(TMG,min_dapi_line = vmin,max_dapi_line=vmax,figsize = (4,4)).show()

In [ ]:
to_keep = np.logical_and(TMG.Layers[0].adata.obs['dapi']>vmin,
                         TMG.Layers[0].adata.obs['dapi']<vmax) 
print(f"before filtering: {TMG.N[0]} cells")
TMG.Layers[0].filter(to_keep)
print(f"after filtering: {TMG.N[0]} cells")

In [ ]:
TMG.Layers[0].build_spatial_graph()
TMG.Layers[0].build_feature_graph(metric='cosine')

In [ ]:
# Create the classifier
optleiden = Classification.OptimalLeidenUnsupervized(TMG.Layers[0])
# train the classifier
optleiden.train(opt_params={'iters':10, 'n_consensus':1})
# use the classifier to create types and add them to TMG using the Taxonomy created on the fly by the classifier
type_vec = optleiden.classify(TMG.Layers[0].feature_mat)

TMG.add_type_information(0, type_vec, optleiden.tax)
TMG.create_isozone_layer()
logging.info(f"TMG has {len(TMG.Layers)} Layers")

In [ ]:
n_topics_list = [2,5,10,15,20,30,50]
n_procs = len(n_topics_list) 

topic_cls = Classification.TopicClassifier(TMG.Layers[0])
topic_cls.train(n_topics_list=n_topics_list, n_procs=n_procs)
topics = topic_cls.classify(topic_cls.Env)

In [ ]:
TMG.create_region_layer(topics, topic_cls.tax)
logging.info(f"TMG has {len(TMG.Layers)} Layers")

In [ ]:
TMG.add_geoms(geom_types = ["mask","voronoi","isozones","regions"],redo=False)

In [ ]:
TMG.save()

In [ ]:
TMG = TissueMultiGraph(inputpath=inputpath,redo = False)

In [ ]:
V = BasisView(TMG,rotation=-90)
V.show()

In [ ]:
SingleMapView(TMG,level_type = "cell", map_type = "random",rotation=-90).show()
SingleMapView(TMG,level_type = "isozone",map_type = "random",rotation=-90).show()
SingleMapView(TMG,level_type = "region",map_type = "random",rotation=-90).show()


In [ ]:
colormaps = ['Purples','Oranges','Blues','Greens','Reds','cividis']
SingleMapView(TMG,level_type = "cell", map_type = "type",color_assign_method = 'linkage',colormaps = colormaps,rotation=-90).show()


In [ ]:
V = UMAPwithSpatialMap(TMG,qntl = (0.025,0.975),clp_embed = (0.025,0.975),rotation=-90)
V.show()